In [ ]:
# Importing the important libraries needed for Data Manipulation, classification and Analysis

import pandas as pd
import numpy as np
import re
from math import isnan
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
from sklearn.datasets import load_digits 
digits = load_digits() 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

# Importing and merging data

In [ ]:
# Import data using predefined file path
data_file1 = 'train.csv' 
df_train= pd.read_csv(data_file1)
# Importing data using user defined file path
df_test = pd.read_csv('test.csv')
df_gender_submission = pd.read_csv('gender_submission.csv')

# Merge the dataset using PassengerId as a key and using right join 
# (all the rows from df_gender_submission and only matching rows from df_test)
result = pd.merge(df_gender_submission, df_test, how = 'right', on = ['PassengerId'])
# Concatenating df_train and result dataframe
train_df = pd.concat([df_train, result], ignore_index=True)

# Initial exploration

In [ ]:
# Check first 3 rows of the data
train_df.head(3) 

In [ ]:
# Print first few rows and last few rows
train_df

In [ ]:
# Check data types
train_df.dtypes

Combining the result from the previous step, when using *head* method of the Dataframe class,
it is clear that all the *object* data types are *Strings*

In [ ]:
# Print number of rows and columns
train_df.shape

In [ ]:
print('\tData description') 
print('*****************************************') 

''' Within the three quatition (i.e ''' '''), it is posible to display a pragraph as it is.
Therefore new line can be enter typing a sentence in a new line or using \n'''
print('''•\tPassengerId: Id of every passenger.\n•\tSurvived: This feature have value 0 and 1. 0 for not survived and 1 for survived.
•\tPclass: There are 3 classes: Class 1, Class 2 and Class 3.\n•\tName: Name of passenger.\n•\tSex: Gender of passenger.
•\tAge: Age of passenger.\n•\tSibSp: Indication that passenger have siblings and spouse.\n•\tParch: Whether a passenger is alone or have family.
•\tTicket: Ticket number of passenger.\n•\tFare: Indicating the fare.\n•\tCabin: The cabin of passenger.\n•\tEmbarked: The embarked category''')

## Exploring data to identify Anomalies/ Missing Data


In [ ]:
# check the unique values count
print('Number of unique values in each columns')
for col in train_df.columns:
    print(str(col)+ '   ' + str(train_df[col].nunique()))
print(f'It is clear when look at the number of unique values, that there are {train_df.PassengerId.nunique()} unique values in the PassengerId column. However, only {train_df.PassengerId.nunique()} unique values in the Name column. This raise the suspision that there may be duplicate entry in the data set')

In [ ]:
#TODO %%%%%%%%%%%%%% Need to write code to check and remove duplicate values

In [ ]:
###insert this plot at the begining of the data when we still had missing values, this is a graphical presentation of the variable where there is missing data in the data set

plt.subplots(figsize=(10,6)) #the figsize help zoom our plot to give a better visible view
sns.heatmap(train_df.isnull(), cbar=False, cmap="YlGnBu_r")

categorical_variable = ['Survived', 'Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked']


In [ ]:
# Percentage of rows missing in each column
for column in train_df.columns:
    percentage = train_df[column].isnull().mean()
    print(f'{column}: {round(percentage*100,4)}%')

In [ ]:
'Our desired number of plot row and column size'
categorical_plot_nrows = 2
categorical_plot_ncols = 3

fig, axs = plt.subplots(categorical_plot_nrows, categorical_plot_ncols, figsize = (categorical_plot_ncols*3.5, categorical_plot_nrows*3))

for r in range(0, categorical_plot_nrows):
    for c in range(0, categorical_plot_ncols):
        i = r*categorical_plot_ncols+c
        ax = axs[r][c]
        sns.countplot(train_df[categorical_variable[i]], hue=train_df["Survived"], ax=ax)
        ax.set_title(categorical_variable[i], fontsize=12, fontweight="bold")
        ax.legend(title="Survived", loc='upper center')

plt.tight_layout()

## Transforming the column values
### Transforming Survived and Pclass columns from integer to String
Values are transformed form numerical values to categotical values as it is easier to understand the data in the exploratary analysis.
Also, after the transformation, graph lables become more discriptive and easier to understand.

### Renaming Embarked column values to their proper Embarcation port
This will make data more understandable in data exploration.


In [ ]:
# Survived column values: maped 1 -> Survived and 0 -> Deceased
train_df.loc[train_df['Survived'] == 0, 'Survived'] = 'Deceased' 
train_df.loc[train_df['Survived'] == 1, 'Survived'] = 'Survived' 
train_df['Survived'].head()

In [ ]:
# Pclass column values: maped 1 -> First Class, 2 -> Second Class and 3 -> Third Class
train_df.loc[train_df['Pclass'] == 1, 'Pclass'] = 'First Class' 
train_df.loc[train_df['Pclass'] == 2, 'Pclass'] = 'Second Class' 
train_df.loc[train_df['Pclass'] == 3, 'Pclass'] = 'Third Class' 
train_df['Pclass'].head()

In [ ]:
# Embarked column values: maped C -> Cherbourg, Q -> Queenstown and S -> Southampton
train_df.loc[train_df['Embarked'] == 'C', 'Embarked'] = 'Cherbourg' 
train_df.loc[train_df['Embarked'] == 'Q', 'Embarked'] = 'Queenstown' 
train_df.loc[train_df['Embarked'] == 'S', 'Embarked'] = 'Southampton'
train_df.head()

In [ ]:
# Can drop PasengerId column as it does not hold valuble information for the analysis
train_df.drop('PassengerId',axis=1)

## Basic statistics before fixing data for missing values

In [ ]:
#The average age of passengers? 
train_df['Age'].mean()

In [ ]:
#TODO%%%%%%%%%%%% WE NEED CHECH THIS @Bassey
# Median Age and Ticket fare price of Pax
#train_df[['Age', 'Fare']].median()
#train_df.describe(include='all')

#instead of using above predifined 'describe function' we can combined aggregating statistics 
train_df.agg( 
    { 
        'Age': ['min', 'max', 'mean', 'median', 'skew'], 
        'Fare': ['min', 'max', 'mean', 'median', 'skew'], 
    } )

In [ ]:
#counting by group
train_df.groupby('Pclass')['Pclass'].count()

In [ ]:
# TODO Do we want all of the below code for our final report@ Bassey
train_df.isnull()  # is any missing values in dataframe
train_df.isnull().any()  # is any missing values across columns
train_df.isnull().sum().sum()   # count of missing values of the entire dataframe
count_NAN = len(train_df) - train_df.count() # count of missing values across columns
count_NAN

max_missing = 0
for i in range(len(train_df.index)):  # count of missing values across rows
    #print('NAN in row ', i , ' : ', train_df.iloc[i].isnull().sum())
    missing_current_row = train_df.iloc[i].isnull().sum()
    if(max_missing < missing_current_row):
        max_missing = missing_current_row

print('maximum null values in a row : ', max_missing, '.','\nTherefore, dropping rows containing missing values are disadvantageous')
print()


In [ ]:
#counting of missing values of a particular column
train_df.Age.isnull().sum()

In [ ]:
# count of missing values of column by group
'''count of missing values of column by group
 Because we dont havve missing values in sex all the missing values is coming from Age'''
train_df.groupby(['Sex'])['Age'].apply(lambda x: x.isnull().sum())

## Fixing missing values

In [ ]:
# Replacing the na in age and Fare column with mean
train_df['Age'] = train_df['Age'].fillna(round(train_df['Age'].mean()))
train_df['Fare'] = train_df['Fare'].fillna(round(train_df['Fare'].mean()))

#checking if there are any remaining missing value
print('Number of missing values remaining in Age column is' ,train_df.Age.isnull().sum())
print('Number of missing values remaining in Fare column is' ,train_df.Fare.isnull().sum())


In [ ]:
# Replace missing values with mode of the Embarked column
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])
print('Number of missing values remaining in Embarked column is' ,train_df.Fare.isnull().sum())

In [ ]:
# See if there is still any missing data in train dataset
print('Missing value remaining in the dataset', train_df.isna().sum().max())
print('Missing values of the Cabin column', train_df.Cabin.isnull().sum())
print('IT is clear that all the remaining missing values are from Cabin column. At his point, not going to impute missing values in Cabin variable as there are more than 60% of the values are missing.')

## Data visualisation

In [ ]:
plt.figure(figsize=(5, 5))
survival_labels = ["Survived", "Deceased"]
survival_counts = [train_df.Survived.value_counts()[1], train_df.Survived.value_counts()[0]]
sns.barplot(x= survival_labels, y = survival_counts)
plt.show()

In [ ]:
# data transformation on Age column
# Grouping age values

train_df.loc[train_df['Age'] <= 1, 'age_group'] = 'baby'
train_df.loc[train_df['Age'].between(1,3), 'age_group'] = 'todlers'
train_df.loc[train_df['Age'].between(4,10), 'age_group'] = 'children'
train_df.loc[train_df['Age'].between(11,19), 'age_group'] = 'teenage'
train_df.loc[train_df['Age'].between(20,35), 'age_group'] = 'yadult'
train_df.loc[train_df['Age'].between(36,60), 'age_group'] = 'adult'
train_df.loc[train_df['Age']>60, 'age_group'] = 'elder'

# Age grouping to two groups Child or Adult
train_df.loc[train_df['Age'] <= 18, 'child_adult'] = 'child'
train_df.loc[train_df['Age'] > 18, 'child_adult'] = 'adult'

# Age grouping using mean values
train_df.loc[train_df['Age'].round() <= 10, 'mean_age'] = 5
train_df.loc[train_df['Age'].round().between(11,20), 'mean_age'] = 15
train_df.loc[train_df['Age'].round().between(21,30), 'mean_age'] = 25
train_df.loc[train_df['Age'].round().between(31,40), 'mean_age'] = 35
train_df.loc[train_df['Age'].round().between(41,50), 'mean_age'] = 45
train_df.loc[train_df['Age'].round().between(51,60), 'mean_age'] = 55
train_df.loc[train_df['Age'].round().between(61,70), 'mean_age'] = 65
train_df.loc[train_df['Age'] > 70, 'mean_age'] = 75

In [ ]:
print('Maximum of Fare: ', df_train['Fare'].max())
print('Minimum of Fare: ', df_train['Fare'].min())
print('Mean of the Fare: ', train_df['Fare'].mean())

# Fare grouping using mean values of the range
train_df.loc[train_df['Fare'].round() <= 100, 'binned_fare'] = 50
train_df.loc[train_df['Fare'].round().between(101,200), 'binned_fare'] = 150
train_df.loc[train_df['Fare'].round().between(201,300), 'binned_fare'] = 250
train_df.loc[train_df['Fare'].round().between(301,400), 'binned_fare'] = 350
train_df.loc[train_df['Fare'].round().between(401,500), 'binned_fare'] = 450
train_df.loc[train_df['Fare'] > 500, 'binned_fare'] = 550

In [ ]:
# Calculate variance using a function

def variance_calculator(df_variable):
    sum_variable = 0
    for value in df_variable:
        if (isnan(value) != True):
            sum_variable += value
    mean = sum_variable/len(df_variable)

 
    sum_of_squares = 0
    for value in df_variable:
        if (isnan(value) != True):
            sum_of_squares += value **2
    mean_squares = sum_of_squares/len(df_variable)


    return mean_squares - mean**2

# The above calculation can also be done using the method numpy.var
# np.var(df_variable)

In [ ]:
print(f'Variance of Fare calculated using the defined function {round(variance_calculator(train_df.Fare),2)}')
print(f'Variance of Fare calculated using numpy function {round(np.var(train_df.Fare),2)}')

Family class with parent and children

SibSp: number of siblings/spouse

Parch: number of childre/parents

With the number of siblings or spouse and the number of children or parents we can create new class called Family.

In [ ]:
# Get pasengers who travel with family (either siblings,spouse, childre or parents)
# Copy rows which has values for either SibSp column or Parch
train_data_families = train_df.loc[(train_df['SibSp'] > 0) | (train_df['Parch'] > 0)][['Survived', 'Name', 'Sex','Age','Ticket','Pclass']]
train_data_families

In [ ]:
def extract_last_name(x):
    # String x will split by ',' and take the first element in the resulting list. 
    # Then any whitespaces at the front and end of the first element will be truncated and returned
    return x.split(",")[0].strip() 

In [ ]:
train_data_families['LastName'] = train_data_families.Name.apply(extract_last_name)
train_data_families.to_csv('family.csv')

### Person Class

Creating a Person class to store pasengers traveling with family.Using the dataset created by filtering rows with non-zero values for variables SibSp(number of siblings/spouse) and Parch(number of childre/parents).

In [ ]:
class Person:
    
    def __init__(self, survival, full_name, sex, age, ticket, last_name):
        self.last_name = last_name
        self.ticket = ticket
        self.full_name = full_name
        self.survival = survival
        self.age = age
        self.sex = sex

In [ ]:
# Create a list to hold person objects
list_of_pasengers = []
'''
This code is written by us. However, as DataFrame object alredy has a similar method, desided to use it to improve performance.
# Create list of tuples with selected column data
with_family_list = list(zip(train_data_families["Survived"], train_data_families["Name"], train_data_families["Sex"],
train_data_families["Age"], train_data_families["Ticket"], train_data_families["LastName"]))

for item in with_family_list:
    p = Person(item[0], item[1], item[2], item[3], item[4], item[5])
    list_of_pasengers.append(p)
'''

for row in train_data_families.itertuples():
    p = Person(row[1], row[2], row[3], row[4], row[5], row[6])
    list_of_pasengers.append(p)

# Just a test
print(list_of_pasengers[6].last_name)
print(list_of_pasengers[6].full_name)
print(list_of_pasengers[6].ticket)
print(list_of_pasengers[6].survival)
print(list_of_pasengers[6].sex)
print(list_of_pasengers[6].age)

In [ ]:
print(len(list_of_pasengers))

When only last names are match to group families, there can be different families with same last names.
Therefore, a variable created with ticket number  and last name used to identify families traveling together.

In [ ]:
# Creating a dictionary to store family members
families_dict = {}
for p in list_of_pasengers:
    # variable containing lastname and ticket number passed as key to the dictionary
    key = f'{p.last_name}_{p.ticket}'
    # If there is already an entry for a family person object will append to the family members list
    # Otherwise new list created with the person object
    if (key in families_dict):
        families_dict[key].append(p)
    else:    
        families_dict[key] = [p]

In [ ]:
print(f'Number of members traveled with family is {len(families_dict)}')

In [ ]:
# create a dictionary for family who traveled as a group
family_group_dict = {}
# If the family has more than one member they are added to another dictionary
for key,value in families_dict.items():
    if len(value) > 1:
        family_group_dict[key] = value

print(f'Number of families traveled together is {len(family_group_dict)}')

Calculate the percentage of adult males and females survived in families

In [ ]:
total_male = 0
total_female = 0
total_children = 0
number_f_adult_s = 0
number_m_adult_s = 0
number_child_s = 0
# Each key value pair represent a family
# Each value in the families_dict is a list family members
for key in family_group_dict:
    #family_mem_list = family_group_dict[key]
    for person in family_group_dict[key]:
        if person.age < 18:
            total_children += 1
            if person.survival == 'Survived':
                number_child_s += 1
        else:
            if (person.sex == "male"):
                total_male += 1
                if(person.survival == 'Survived'):
                    number_m_adult_s += 1
            else:
                total_female +=1
                if(person.survival == 'Survived'):
                    number_f_adult_s += 1

print('When a pasenger traveled as a family')
print("percentage of females survived " + str(round(100 * number_f_adult_s/total_female)) + '%')
print("percentage of males survived " + str(round(100 * number_m_adult_s/total_male)) + '%')
print("percentage of children survived " + str(round(100 * number_child_s/total_children)) + '%')


## Building a Random Forest Classifier
Reference: https://www.freecodecamp.org/news/how-to-use-the-tree-based-algorithm-for-machine-learning/

### Random Forest Classifier with variables
- Pclass
- Sex
- SibSp
- Parch
- mean_age

In [ ]:
# Create a copy of the dataset because some columns are transforming to numeric
rf_df = train_df
# Convert variables to numeric values
rf_df['Sex'] = rf_df['Sex'].replace(['female','male'],[0,1])

train_df.loc[train_df['Survived'] == 'Deceased', 'Survived'] = 0 
train_df.loc[train_df['Survived'] == 'Survived', 'Survived'] = 1

train_df.loc[train_df['Pclass'] == 'First Class', 'Pclass'] = 1
train_df.loc[train_df['Pclass'] == 'Second Class', 'Pclass'] = 2 
train_df.loc[train_df['Pclass'] == 'Third Class' , 'Pclass'] = 3

rf_df['Survived'] = rf_df['Survived'].astype('int')
rf_df['Pclass'] = rf_df['Pclass'].astype('int')
rf_df['mean_age'] = rf_df['mean_age'].astype('int')

In [ ]:
# split data into input and taget variable(s)
X = rf_df.drop(["Survived", 'Name', 'Age', 'Ticket', 'Fare', 'PassengerId', 'Cabin', 'Embarked', 'age_group', 'child_adult', 'binned_fare'], axis = 1)
y = rf_df["Survived"]
X

In [ ]:
# standardize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, stratify=y, test_size=0.10, random_state=42
)

In [ ]:
# create the classifier
classifier = RandomForestClassifier(n_estimators=100)

# Train the model using the training sets
classifier.fit(X_train, y_train)

In [ ]:
# predictin on the test set
y_pred = classifier.predict(X_test)

In [ ]:
# Calculate Model Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))